# Creating datasets


This is probably the most interesting part of the tutorial, where you get to leverage EOTDL tools to create a brand new dataset. Here we cover:

1. **Data exploration**: given an area of interest, query available sentinel data for your dataset.
2. **Data access**: download your data for creating the dataset.
3. **Data preparation**: clean your data, perform feature engineering, data analysis, labelling, etc.

Once your dataset is ready, you can ingest it to the EOTDL like we have seen in the previous notebook and start working with it as any other dataset in the repository.


## Exploration


First of all, let's explore the area of interest that we have selected for this workshop. In this case we have chosen the [Boadella reservoir](https://es.wikipedia.org/wiki/Embalse_de_Darnius_Boadella) in Catalonia, Spain, which geometry is in the data folder as `workshop_data/boadella.geojson`. Here we use [leafmap](https://leafmap.org/) for visualizing it, but feel free to use your preferred solution.


In [1]:
# !pip install leafmap

In [4]:
import leafmap
import geopandas as gpd

in_geojson = "workshop_data/boadella.geojson"
gdf = gpd.read_file(in_geojson)

centroid_coords = gdf["geometry"].centroid
centroid = [
    centroid_coords.y.values[0],
    centroid_coords.x.values[0],
]  # We are going to use the centroid later

m = leafmap.Map(center=centroid, zoom=13)
m.add_geojson(in_geojson, layer_name="Boadella reservoir")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

When creating AI-Ready datasets it is usual to work at a fixed resolution. You can either retrieve full scenes and cut patches, or use EOTDL functionality to generate appropriate bounding boxes. With the aim that all the images in the dataset have 512x512 pixels, we are going to use the centroid that we extracted before from the geoJSON and generate a bounding box that will result in a 512x512 pixels image at 10m resolution since we are going to use Sentinel data.


In [5]:
from eotdl.tools import bbox_from_centroid

boadella_bbox = bbox_from_centroid(
    x=centroid[0], y=centroid[1], pixel_size=10, width=512, height=512
)
boadella_bbox

[2.7920278066359443, 42.33057868499878, 2.8380215491828635, 42.36457137143556]

Let's visualize the bounding box on a map!


In [6]:
from eotdl.tools import bbox_to_polygon

# Create a polygon from the bbox
boadella_polygon = bbox_to_polygon(boadella_bbox)
# Create a GeoDataFrame from the polygon
gdf = gpd.GeoDataFrame(geometry=[boadella_polygon])
# Save the bounding box as a geoJSON file, if needed
gdf.to_file(
    "workshop_data/boadella_bbox.geojson", driver="GeoJSON"
)  # Uncomment to save the bbox as a GeoJSON file

m.add_geojson("workshop_data/boadella_bbox.geojson", layer_name="Boadella bbox")
m

Map(bottom=776066.0, center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position'…

Now that he have our desired bounding box we can look for available Sentinel-2 imagery on it. This can be done through the EOTDL.

First, we can look for which Sentinel sensors are supported in the EOTDL


In [1]:
from eotdl.access import SUPPORTED_SENSORS

SUPPORTED_SENSORS

('sentinel-1-grd', 'sentinel-2-l1c', 'sentinel-2-l2a', 'dem')

If we want to look for available Sentinel-2 imagery in our AoI, we must define a range of dates in which to search for the images. We have already defined a time interval for this workshop, which is in the `workshop_data/dates.csv` file.


In [2]:
import csv

dates = list()
with open("workshop_data/dates.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        dates.append(row[0])
dates.sort()

dates[:5]

['2020-01-13', '2020-01-28', '2020-02-02', '2020-06-21', '2020-09-14']

Although we have the specific dates, we are going to search for the entire time interval, just as a demonstrator.


> We use Sentinle Hub under the hood, so you will need appropriate credentials. You can generate them automatically from your user [profile](https://www.eotdl.com/profile) by accepting the terms and conditions. When you login to the EOTDL, via the library or CLI, we retrieve and store this information for you, so you don't need to worry about it. However there are a couple of gotchas: <br><br> 1. If you already have a Sentinel HUB account with the same email as your EOTDL account, you will need to retrieve the credentials from Sentinel Hub Dashboard and set them as env variables. <br> 2. The credentials generated via EOTDL may expire after some time (we are working on this). If this happens, let us know in Discord to fix the issue. <br><br> In any case, you can provide your own credentials by setting the appropriate environment variables: `SH_CLIENT_ID` and `SH_CLIENT_SECRET`.


In [7]:
from eotdl.access import search_sentinel_imagery

time_interval = (dates[0], dates[-1])

r = search_sentinel_imagery(time_interval, boadella_bbox, "sentinel-2-l2a")
response = list(r)
response[:5]

[{'id': 'S2B_MSIL2A_20220601T103629_N0400_R008_T31TDG_20220601T135543',
  'properties': {'datetime': '2022-06-01T10:49:26Z', 'eo:cloud_cover': 0.23}},
 {'id': 'S2B_MSIL2A_20220601T103629_N0400_R008_T31TDH_20220601T135543',
  'properties': {'datetime': '2022-06-01T10:49:14Z', 'eo:cloud_cover': 12.82}},
 {'id': 'S2A_MSIL2A_20220527T103631_N0400_R008_T31TDG_20220527T183616',
  'properties': {'datetime': '2022-05-27T10:49:34Z', 'eo:cloud_cover': 85.6}},
 {'id': 'S2A_MSIL2A_20220527T103631_N0400_R008_T31TDH_20220527T183616',
  'properties': {'datetime': '2022-05-27T10:49:19Z', 'eo:cloud_cover': 30.42}},
 {'id': 'S2B_MSIL2A_20220522T103629_N0400_R008_T31TDG_20220522T124154',
  'properties': {'datetime': '2022-05-22T10:49:27Z', 'eo:cloud_cover': 12.99}}]

They make sense, as the [revisit time](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/#basic-facts) for Sentinel-2 is 5 days.

As a final step, let's check the number of dates with available images.


In [8]:
print(len(response))

342


To sum up this section, we have explored our AoI, generated a bounding box and a time interval in which to look for imagery and searched for Sentinel-2 imagery.


## Download


The next step is to download the images. On the one hand, we can download image by image, as follows.


In [9]:
from eotdl.access import download_sentinel_imagery

first_date = dates[0]

# Uncomment to demonstrate
# download_sentinel_imagery(
#     data/sentinel_2", first_date, boadella_bbox, "sentinel-2-l2a"
# )

On the other hand, we can search and download all available images within a time interval, as follows. This is the recommended way for a bulk download, but it has the drawback that we cannot control the quality of the images, as for example know their cloud cover.


In [11]:
# Uncomment to demonstrate
# demostration_dates = (dates[0], dates[2])
# download_sentinel_imagery(
#     output="data/sentinel_2",
#     time_interval=demostration_dates,
#     bounding_box=boadella_bbox,
#     sensor="sentinel-2-l2a",
# )

Despite what we have seen, in the `workshop_data/dates.csv` file we already have a list with the acquisition dates of valid, cloud-free and good quality images. This is a slower but safer solution. So, let's download them!


In [12]:
for date in dates:
    download_sentinel_imagery("data/sentinel_2", date, boadella_bbox, "sentinel-2-l2a")

That's all! We have downloaded the images for our dataset. Let's check them!


In [11]:
from glob import glob

rasters = glob("data/sentinel_2/*.tif")
rasters[:5]

['data/sentinel_2/sentinel-2-l2a_2022-06-01.tif',
 'data/sentinel_2/sentinel-2-l2a_2022-03-08.tif',
 'data/sentinel_2/sentinel-2-l2a_2021-07-21.tif',
 'data/sentinel_2/sentinel-2-l2a_2020-01-23.tif',
 'data/sentinel_2/sentinel-2-l2a_2020-02-02.tif']

We can look for them metadata files, too.


In [12]:
jsons = glob("data/sentinel_2/*.json")
jsons[:5]

['data/sentinel_2/sentinel-2-l2a_2020-02-02.json',
 'data/sentinel_2/sentinel-2-l2a_2021-07-16.json',
 'data/sentinel_2/sentinel-2-l2a_2022-06-01.json',
 'data/sentinel_2/sentinel-2-l2a_2020-06-21.json',
 'data/sentinel_2/sentinel-2-l2a_2020-01-13.json']

It looks amazing! One last step, in order to easily ingest the images generate STAC metadata in next steps, we will rename the images, maintaining the acquisiton date but replacing the sensor type in the filename by `Boadella`, and delete the metadata files. This is not mandatory, but it will be useful for our usecase.


In [13]:
files = glob('data/sentinel_2/*')
for file in files:
    new_file_name = file.replace('sentinel-2-l2a', 'Boadella')
    ! mv $file $new_file_name

!rm -r data/sentinel_2/*.json

## Data Preparation


As the final step towards creating our training dataset, we need to make the data AI-Ready. There are multitude of tasks that can be performed here, such as:

- **Data cleaning**: remove corrupted images, remove images with too much cloud cover, etc.
- **Feature engineering**: calculate vegetation indices, calculate statistics, etc.
- **Data analysis**: plot time series, plot histograms, etc.
- **Labelling**: create labels for the images, etc.

For each one, feel free to use your favourite tools. Here we are going to demonstrate labelling using SCANEO.

SCANEO is a labelling web application that allows tagging satellite images (to identify, e.g., objects present, terrain types, etc.) in an easy and fast way. The service provided by SCANEO is vital since it is necessary to prepare the satellite data so that it can be processed by neural networks, enabling active learning.

Before running the web interface, we need to make sure we have the `scaneo` package installed in our machine and, if not, install it.


In [15]:
# !pip install scaneo

You can run `scaneo` with the following options


In [17]:
!scaneo --help

                                                                                
 Usage: scaneo [OPTIONS]                                                        
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --port                -p      INTEGER  Port to run the server on             │
│                                        [default: 8000]                       │
│ --host                -h      TEXT     Host to run the server on             │
│                                        [default: localhost]                  │
│ --data                -d      PATH     Path to data directory                │
│                                        [default: None]                       │
│ --image               -i      PATH     Save masks as vector or raster        │
│                                        [default: vector]                     │
│ --env                 -e  

As seen, we have several options in `scaneo` usage, such as selecting the default port to run the server, the host, environment parameters, and so on. In this workshop, what we need is as simple as give the path of our dataset. You can launch `scaneo` with the following command: `scaneo --data data/sentinel_2`


![scaneo](./images/scaneo.png)


Your annotations will be stored alongside the images as GeoJSON files containig the segmentation masks as multipolygons, bounding boxes for detection tasks or classification labels.


In [19]:
!ls data/sentinel_2/*.geojson

data/sentinel_2/Boadella_2020-01-28_labels.geojson


Once your data is ready you can ingest it to EOTDL like we have seen in the previous notebook and start working with it as any other dataset in the repository.


In [18]:
import yaml

metadata = {
    "authors": ["Fran Martin", "Juan B. Pedro"],
    "license": "free",
    "source": "https://github.com/earthpulse/eotdl/blob/develop/tutorials/workshops/bids23/04_creating.ipynb",
    "name": "Boadella-BiDS23",
}

with open("workshop_data/sentinel_2/metadata.yml", "w") as outfile:
    yaml.dump(metadata, outfile, default_flow_style=False)

In [20]:
text = """---
name: Boadella-PhiLab24
authors: 
  - Juan B. Pedro
  - Fran Martin
license: free
source: https://github.com/earthpulse/eotdl/blob/develop/tutorials/workshops/philab24/04_creating.ipynb
---

# Boadella-PhiLab24

This is a toy dataset created during the PhiLab24 workshop.
"""

with open("data/sentinel_2/README.md", "w") as outfile:
    outfile.write(text)

In [21]:
from eotdl.datasets import ingest_dataset

ingest_dataset("data/sentinel_2")

Uploading directory data/sentinel_2...
generating list of files to upload...


100%|██████████| 14/14 [00:00<00:00, 448.09it/s]

14 new files will be ingested
0 files already exist in dataset
0 large files will be ingested separately


New version created, version: 1
generating batches...


100%|██████████| 14/14 [00:00<00:00, 92327.45it/s]


Uploading 14 small files in 2 batches...


Uploading batches: 100%|██████████| 2/2 [00:03<00:00,  1.54s/batches]


{'dataset_id': '6630be393a7b02c7f6c4592b',
 'dataset_name': 'Boadella-PhiLab24',
 'filenames': ['Boadella_2020-01-13.tif',
  'Boadella_2020-06-21.tif',
  'Boadella_2022-06-01.tif',
  'Boadella_2020-01-28_labels.geojson',
  'labels.json',
  'Boadella_2020-02-02.tif',
  'Boadella_2021-07-21.tif',
  'Boadella_2020-01-23.tif']}

If you add more images or labels to the dataset, you can re-upload and a new version will be automatically generated.

However, you might want to wait for the next tutorial where you will find how to generate STAC metadata for this dataset in order to ingest it to the EOTDL as Q1 or Q2 datasets, leveraging advanced functionality.


## Discussion and Contribution opportunities


Feel free to ask questions now (live or through Discord) and make suggestions for future improvements.

- What features concerning data exploration would you like to see?
- What other features concerning data download would you like to see?
- What features and tools concerning data preparation would you like to see?
- What does your typical workflow look like?
- Do you already use any labelling tool?
- What does you ideal labelling tool looks like?
